# HealthCost Insights: Healthcare Billing Anomaly Detection 🏥💰

## Project Overview
This notebook provides comprehensive analysis of healthcare billing data to detect anomalies, identify cost patterns, and prepare insights for Power BI dashboards. The analysis includes:

- **Data Generation**: Creating realistic healthcare billing datasets
- **Exploratory Analysis**: Understanding data patterns and distributions  
- **Anomaly Detection**: Statistical and ML-based approaches to identify billing irregularities
- **Cost Analysis**: Deep dive into cost patterns and trends
- **Visualization**: Interactive charts and plots for insights
- **Export Preparation**: Clean data export for Power BI integration

## Business Value
Healthcare billing anomaly detection can help organizations:
- 🔍 Identify potential fraud and billing errors
- 💡 Optimize cost management and resource allocation
- 📊 Improve financial transparency and compliance
- ⚡ Enhance operational efficiency in revenue cycle management

---

## 1. Import Required Libraries 📚

Importing essential libraries for data manipulation, analysis, visualization, and machine learning.

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine learning and statistical analysis
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import zscore

# Utility libraries
import warnings
import os
from faker import Faker
import random

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
fake = Faker()
Faker.seed(42)

print("✅ All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🤖 Scikit-learn version: {sklearn.__version__}")

## 2. Generate Simulated Healthcare Billing Dataset 🏥

Creating a comprehensive synthetic healthcare billing dataset with realistic features and intentional anomalies for analysis.

In [ ]:
def generate_healthcare_billing_data(num_records=50000):
    """
    Generate comprehensive healthcare billing dataset with realistic anomalies
    """
    print(f"🔧 Generating {num_records:,} healthcare billing records...")
    
    # Define realistic medical procedures with base costs and frequencies
    procedures = {
        'Emergency Room Visit': {'base_cost': 1200, 'variance': 400, 'frequency': 0.15},
        'Routine Checkup': {'base_cost': 250, 'variance': 50, 'frequency': 0.25},
        'Blood Test': {'base_cost': 150, 'variance': 30, 'frequency': 0.20},
        'X-Ray': {'base_cost': 300, 'variance': 75, 'frequency': 0.12},
        'MRI Scan': {'base_cost': 2500, 'variance': 500, 'frequency': 0.05},
        'CT Scan': {'base_cost': 1800, 'variance': 300, 'frequency': 0.08},
        'Surgery - Minor': {'base_cost': 5000, 'variance': 1000, 'frequency': 0.04},
        'Surgery - Major': {'base_cost': 25000, 'variance': 8000, 'frequency': 0.02},
        'Physical Therapy': {'base_cost': 180, 'variance': 40, 'frequency': 0.09}
    }
    
    # Insurance providers with coverage characteristics
    insurance_providers = {
        'BlueCross BlueShield': {'coverage_rate': 0.80, 'frequency': 0.25},
        'Aetna': {'coverage_rate': 0.75, 'frequency': 0.20},
        'UnitedHealth': {'coverage_rate': 0.82, 'frequency': 0.22},
        'Cigna': {'coverage_rate': 0.78, 'frequency': 0.15},
        'Medicare': {'coverage_rate': 0.85, 'frequency': 0.10},
        'Medicaid': {'coverage_rate': 0.90, 'frequency': 0.08}
    }
    
    # Medical departments and specialties
    departments = [
        'Emergency Medicine', 'Internal Medicine', 'Cardiology', 'Orthopedics',
        'Radiology', 'Surgery', 'Pediatrics', 'Neurology', 'Oncology', 'Psychiatry'
    ]
    
    # Generate patient demographics and billing data
    data = []
    
    for i in range(num_records):
        # Patient information
        patient_id = f"P{10000 + i:06d}"
        patient_age = max(1, min(95, int(np.random.normal(45, 18))))
        patient_gender = np.random.choice(['Male', 'Female', 'Other'], p=[0.48, 0.50, 0.02])
        
        # Procedure selection based on realistic frequencies
        procedure_name = np.random.choice(
            list(procedures.keys()),
            p=[procedures[p]['frequency'] for p in procedures.keys()]
        )
        procedure_info = procedures[procedure_name]
        
        # Calculate realistic billing amounts
        base_cost = max(50, np.random.normal(
            procedure_info['base_cost'], 
            procedure_info['variance']
        ))
        
        # Age-based cost adjustment (older patients often have higher costs)
        age_multiplier = 1.0 + (patient_age - 30) * 0.005 if patient_age > 30 else 1.0
        base_cost *= age_multiplier
        
        # Insurance selection and payment calculation
        insurance_name = np.random.choice(
            list(insurance_providers.keys()),
            p=[insurance_providers[p]['frequency'] for p in insurance_providers.keys()]
        )
        coverage_rate = insurance_providers[insurance_name]['coverage_rate']
        
        # Calculate payment amounts with realistic variation
        total_billed = base_cost
        insurance_paid = total_billed * coverage_rate * np.random.uniform(0.85, 1.0)
        patient_responsibility = max(0, total_billed - insurance_paid)
        
        # Generate service dates (last 24 months)
        start_date = datetime.now() - timedelta(days=730)
        service_date = start_date + timedelta(days=random.randint(0, 730))
        
        # Additional realistic fields
        department = random.choice(departments)
        provider_id = f"DR{random.randint(1000, 9999)}"
        
        # Diagnosis codes (simplified ICD-10 style)
        diagnosis_codes = [
            'Z00.00', 'I10', 'E11.9', 'M79.1', 'R53.83', 'K21.9',
            'F41.1', 'M25.511', 'N39.0', 'R50.9', 'H52.4', 'J06.9'
        ]
        primary_diagnosis = random.choice(diagnosis_codes)
        
        # Admission and length of stay logic
        if 'Surgery' in procedure_name or 'Emergency' in procedure_name:
            admission_type = np.random.choice(['Inpatient', 'Emergency'], p=[0.7, 0.3])
            length_of_stay = random.randint(1, 10)
        else:
            admission_type = 'Outpatient'
            length_of_stay = 1
        
        data.append({
            'claim_id': f"CLM{20240000 + i:08d}",
            'patient_id': patient_id,
            'patient_age': patient_age,
            'patient_gender': patient_gender,
            'service_date': service_date.strftime('%Y-%m-%d'),
            'procedure_name': procedure_name,
            'procedure_code': f"CPT{random.randint(10000, 99999)}",
            'primary_diagnosis': primary_diagnosis,
            'department': department,
            'provider_id': provider_id,
            'insurance_provider': insurance_name,
            'total_billed_amount': round(total_billed, 2),
            'insurance_paid_amount': round(insurance_paid, 2),
            'patient_responsibility': round(patient_responsibility, 2),
            'claim_status': np.random.choice(['Paid', 'Pending', 'Denied'], p=[0.85, 0.10, 0.05]),
            'admission_type': admission_type,
            'length_of_stay': length_of_stay,
            'facility_type': np.random.choice(['Hospital', 'Clinic', 'Urgent Care'], p=[0.6, 0.3, 0.1])
        })
    
    df = pd.DataFrame(data)
    
    # Introduce realistic anomalies (approximately 5% of data)
    print("🚨 Introducing realistic billing anomalies...")
    anomaly_count = int(num_records * 0.05)
    anomaly_indices = np.random.choice(df.index, anomaly_count, replace=False)
    
    anomaly_types = []
    
    for idx in anomaly_indices:
        anomaly_type = np.random.choice([
            'billing_error', 'duplicate_claim', 'unusual_cost', 'fraud_pattern'
        ], p=[0.3, 0.25, 0.25, 0.2])
        
        anomaly_types.append(anomaly_type)
        
        if anomaly_type == 'billing_error':
            # Billing calculation errors - inflate costs unreasonably
            df.loc[idx, 'total_billed_amount'] *= np.random.uniform(3.0, 8.0)
            
        elif anomaly_type == 'duplicate_claim':
            # Create potential duplicate claims
            if idx < len(df) - 1:
                df.loc[idx + 1, 'patient_id'] = df.loc[idx, 'patient_id']
                df.loc[idx + 1, 'procedure_name'] = df.loc[idx, 'procedure_name']
                df.loc[idx + 1, 'service_date'] = df.loc[idx, 'service_date']
                df.loc[idx + 1, 'provider_id'] = df.loc[idx, 'provider_id']
                
        elif anomaly_type == 'unusual_cost':
            # Routine procedures with unusually high costs
            if 'Routine' in df.loc[idx, 'procedure_name'] or 'Blood Test' in df.loc[idx, 'procedure_name']:
                df.loc[idx, 'total_billed_amount'] *= np.random.uniform(15.0, 30.0)
                
        elif anomaly_type == 'fraud_pattern':
            # Potential fraud indicators - multiple expensive procedures
            df.loc[idx, 'procedure_name'] = 'Surgery - Major'
            df.loc[idx, 'total_billed_amount'] = np.random.uniform(80000, 150000)
            df.loc[idx, 'length_of_stay'] = random.randint(15, 30)
    
    # Add calculated fields for analysis
    df['payment_rate'] = df['insurance_paid_amount'] / df['total_billed_amount']
    df['cost_per_day'] = df['total_billed_amount'] / df['length_of_stay']
    df['service_month'] = pd.to_datetime(df['service_date']).dt.to_period('M').astype(str)
    df['service_quarter'] = pd.to_datetime(df['service_date']).dt.quarter
    df['service_year'] = pd.to_datetime(df['service_date']).dt.year
    df['day_of_week'] = pd.to_datetime(df['service_date']).dt.day_name()
    
    print(f"✅ Generated {len(df):,} healthcare billing records with {anomaly_count:,} anomalies")
    return df

# Generate the dataset
healthcare_df = generate_healthcare_billing_data(50000)

print(f"\n📋 Dataset Shape: {healthcare_df.shape}")
print(f"📅 Date Range: {healthcare_df['service_date'].min()} to {healthcare_df['service_date'].max()}")
print(f"💰 Total Billed: ${healthcare_df['total_billed_amount'].sum():,.2f}")
print(f"👥 Unique Patients: {healthcare_df['patient_id'].nunique():,}")
print(f"👨‍⚕️ Unique Providers: {healthcare_df['provider_id'].nunique():,}")

## 3. Data Exploration and Profiling 🔍

Comprehensive exploratory data analysis to understand the dataset structure, quality, and initial patterns.

In [ ]:
# Basic dataset information
print("📊 DATASET OVERVIEW")
print("=" * 50)
print(f"Dataset Shape: {healthcare_df.shape}")
print(f"Memory Usage: {healthcare_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Duplicate Rows: {healthcare_df.duplicated().sum():,}")

# Data types and missing values
print("\n🔍 DATA QUALITY ASSESSMENT")
print("=" * 50)
info_df = pd.DataFrame({
    'Column': healthcare_df.columns,
    'Data_Type': healthcare_df.dtypes,
    'Non_Null_Count': healthcare_df.count(),
    'Null_Count': healthcare_df.isnull().sum(),
    'Null_Percentage': (healthcare_df.isnull().sum() / len(healthcare_df) * 100).round(2),
    'Unique_Values': healthcare_df.nunique()
})
print(info_df)

# Statistical summary for numerical columns
print("\n📈 NUMERICAL COLUMNS STATISTICS")
print("=" * 50)
numerical_cols = ['patient_age', 'total_billed_amount', 'insurance_paid_amount', 
                 'patient_responsibility', 'length_of_stay', 'cost_per_day', 'payment_rate']
print(healthcare_df[numerical_cols].describe())

# Categorical columns overview
print("\n📋 CATEGORICAL COLUMNS OVERVIEW")
print("=" * 50)
categorical_cols = ['procedure_name', 'insurance_provider', 'department', 
                   'claim_status', 'admission_type', 'patient_gender']

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print(healthcare_df[col].value_counts().head())

# Display sample records
print("\n📝 SAMPLE RECORDS")
print("=" * 50)
print(healthcare_df.head())

# Check for potential data quality issues
print("\n⚠️ POTENTIAL DATA QUALITY ISSUES")
print("=" * 50)

# Negative amounts
negative_amounts = healthcare_df[healthcare_df['total_billed_amount'] < 0]
print(f"Records with negative billing amounts: {len(negative_amounts)}")

# Zero amounts
zero_amounts = healthcare_df[healthcare_df['total_billed_amount'] == 0]
print(f"Records with zero billing amounts: {len(zero_amounts)}")

# Payment rate anomalies (>100% or <0%)
payment_anomalies = healthcare_df[
    (healthcare_df['payment_rate'] > 1.0) | (healthcare_df['payment_rate'] < 0)
]
print(f"Records with unusual payment rates: {len(payment_anomalies)}")

# Very high billing amounts (potential outliers)
high_amounts = healthcare_df[healthcare_df['total_billed_amount'] > 100000]
print(f"Records with billing amounts > $100,000: {len(high_amounts)}")

print(f"\n✅ Data exploration completed successfully!")

## 4. Data Cleaning and Preprocessing 🧹

Cleaning the dataset and creating additional features for enhanced analysis.

In [ ]:
# Create a working copy for cleaning
df_clean = healthcare_df.copy()

print("🧹 STARTING DATA CLEANING PROCESS")
print("=" * 50)
print(f"Initial dataset size: {df_clean.shape}")

# 1. Handle missing values (if any)
print(f"\nMissing values before cleaning: {df_clean.isnull().sum().sum()}")

# 2. Remove records with negative or zero billing amounts
initial_count = len(df_clean)
df_clean = df_clean[df_clean['total_billed_amount'] > 0]
removed_count = initial_count - len(df_clean)
print(f"Removed {removed_count} records with invalid billing amounts")

# 3. Cap extremely high billing amounts (potential outliers)
# Using 99.5th percentile as threshold
billing_threshold = df_clean['total_billed_amount'].quantile(0.995)
extreme_outliers = df_clean[df_clean['total_billed_amount'] > billing_threshold]
print(f"Found {len(extreme_outliers)} extreme outliers (>${billing_threshold:,.2f})")

# 4. Convert date columns to datetime
df_clean['service_date'] = pd.to_datetime(df_clean['service_date'])

# 5. Create additional derived features for analysis
print(f"\n🔧 CREATING DERIVED FEATURES")
print("=" * 30)

# Age groups for demographic analysis
def categorize_age(age):
    if age < 18:
        return 'Pediatric (0-17)'
    elif age < 35:
        return 'Young Adult (18-34)'
    elif age < 50:
        return 'Middle Age (35-49)'
    elif age < 65:
        return 'Older Adult (50-64)'
    else:
        return 'Senior (65+)'

df_clean['age_group'] = df_clean['patient_age'].apply(categorize_age)

# Cost categories
def categorize_cost(amount):
    if amount < 500:
        return 'Low Cost (<$500)'
    elif amount < 2000:
        return 'Medium Cost ($500-$2,000)'
    elif amount < 10000:
        return 'High Cost ($2,000-$10,000)'
    else:
        return 'Very High Cost (>$10,000)'

df_clean['cost_category'] = df_clean['total_billed_amount'].apply(categorize_cost)

# Procedure complexity based on cost
df_clean['procedure_complexity'] = df_clean['procedure_name'].map({
    'Emergency Room Visit': 'High',
    'Routine Checkup': 'Low',
    'Blood Test': 'Low',
    'X-Ray': 'Medium',
    'MRI Scan': 'High',
    'CT Scan': 'High',
    'Surgery - Minor': 'High',
    'Surgery - Major': 'Very High',
    'Physical Therapy': 'Medium'
})

# Days between service and today (recency)
df_clean['days_since_service'] = (pd.Timestamp.now() - df_clean['service_date']).dt.days

# Insurance efficiency (how much insurance covers)
df_clean['insurance_efficiency'] = df_clean['insurance_paid_amount'] / df_clean['total_billed_amount']

# Weekend indicator
df_clean['is_weekend'] = df_clean['service_date'].dt.weekday >= 5

# Monthly billing totals per patient
monthly_patient_totals = df_clean.groupby(['patient_id', 'service_month'])['total_billed_amount'].sum().reset_index()
monthly_patient_totals.columns = ['patient_id', 'service_month', 'monthly_patient_total']
df_clean = df_clean.merge(monthly_patient_totals, on=['patient_id', 'service_month'], how='left')

# Provider billing statistics
provider_stats = df_clean.groupby('provider_id').agg({
    'total_billed_amount': ['count', 'mean', 'sum'],
    'patient_id': 'nunique'
}).round(2)
provider_stats.columns = ['provider_claim_count', 'provider_avg_billing', 'provider_total_billing', 'provider_unique_patients']
provider_stats = provider_stats.reset_index()
df_clean = df_clean.merge(provider_stats, on='provider_id', how='left')

print(f"✅ Added {len([col for col in df_clean.columns if col not in healthcare_df.columns])} new derived features")

# 6. Identify potential duplicate claims
print(f"\n🔍 IDENTIFYING POTENTIAL DUPLICATE CLAIMS")
print("=" * 40)

# Define criteria for potential duplicates
duplicate_criteria = ['patient_id', 'procedure_name', 'service_date', 'provider_id']
potential_duplicates = df_clean[df_clean.duplicated(subset=duplicate_criteria, keep=False)]
print(f"Found {len(potential_duplicates)} potential duplicate claims")

# Flag duplicates without removing them
df_clean['is_potential_duplicate'] = df_clean.duplicated(subset=duplicate_criteria, keep=False)

# 7. Create final summary
print(f"\n📊 FINAL DATASET SUMMARY")
print("=" * 30)
print(f"Final dataset size: {df_clean.shape}")
print(f"Total features: {len(df_clean.columns)}")
print(f"Date range: {df_clean['service_date'].min().date()} to {df_clean['service_date'].max().date()}")
print(f"Total billing amount: ${df_clean['total_billed_amount'].sum():,.2f}")
print(f"Average claim amount: ${df_clean['total_billed_amount'].mean():,.2f}")
print(f"Potential duplicates flagged: {df_clean['is_potential_duplicate'].sum():,}")

print(f"\n✅ Data cleaning and preprocessing completed successfully!")

## 5. Statistical Anomaly Detection 📊

Implementing statistical methods to identify billing anomalies and outliers using Z-score, IQR, and percentile-based approaches.

In [ ]:
def detect_statistical_anomalies(df, target_column='total_billed_amount'):
    """
    Detect anomalies using multiple statistical methods
    """
    print(f"🔍 STATISTICAL ANOMALY DETECTION FOR: {target_column}")
    print("=" * 60)
    
    anomaly_results = {}
    
    # 1. Z-Score Method (threshold: |z| > 3)
    z_scores = np.abs(zscore(df[target_column]))
    z_anomalies = z_scores > 3
    anomaly_results['z_score'] = z_anomalies
    
    print(f"📊 Z-Score Method (|z| > 3):")
    print(f"   Anomalies detected: {z_anomalies.sum():,} ({z_anomalies.mean()*100:.2f}%)")
    print(f"   Z-score range: {z_scores.min():.2f} to {z_scores.max():.2f}")
    
    # 2. Interquartile Range (IQR) Method
    Q1 = df[target_column].quantile(0.25)
    Q3 = df[target_column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_anomalies = (df[target_column] < lower_bound) | (df[target_column] > upper_bound)
    anomaly_results['iqr'] = iqr_anomalies
    
    print(f"\n📦 IQR Method (1.5 × IQR):")
    print(f"   Lower bound: ${lower_bound:,.2f}")
    print(f"   Upper bound: ${upper_bound:,.2f}")
    print(f"   Anomalies detected: {iqr_anomalies.sum():,} ({iqr_anomalies.mean()*100:.2f}%)")
    
    # 3. Percentile Method (99th percentile)
    percentile_99 = df[target_column].quantile(0.99)
    percentile_1 = df[target_column].quantile(0.01)
    
    percentile_anomalies = (df[target_column] > percentile_99) | (df[target_column] < percentile_1)
    anomaly_results['percentile'] = percentile_anomalies
    
    print(f"\n📈 Percentile Method (1st/99th percentiles):")
    print(f"   1st percentile: ${percentile_1:,.2f}")
    print(f"   99th percentile: ${percentile_99:,.2f}")
    print(f"   Anomalies detected: {percentile_anomalies.sum():,} ({percentile_anomalies.mean()*100:.2f}%)")
    
    # 4. Modified Z-Score (using median absolute deviation)
    median = df[target_column].median()
    mad = np.median(np.abs(df[target_column] - median))
    modified_z_scores = 0.6745 * (df[target_column] - median) / mad
    modified_z_anomalies = np.abs(modified_z_scores) > 3.5
    anomaly_results['modified_z'] = modified_z_anomalies
    
    print(f"\n🎯 Modified Z-Score Method (MAD-based):")
    print(f"   Median: ${median:,.2f}")
    print(f"   MAD: ${mad:,.2f}")
    print(f"   Anomalies detected: {modified_z_anomalies.sum():,} ({modified_z_anomalies.mean()*100:.2f}%)")
    
    # Combine all methods - consensus approach
    anomaly_counts = sum(anomaly_results.values())
    consensus_anomalies = anomaly_counts >= 2  # At least 2 methods agree
    anomaly_results['consensus'] = consensus_anomalies
    
    print(f"\n🤝 Consensus Method (≥2 methods agree):")
    print(f"   Anomalies detected: {consensus_anomalies.sum():,} ({consensus_anomalies.mean()*100:.2f}%)")
    
    return anomaly_results

# Apply statistical anomaly detection to billing amounts
billing_anomalies = detect_statistical_anomalies(df_clean, 'total_billed_amount')

# Add anomaly flags to dataframe
for method, anomalies in billing_anomalies.items():
    df_clean[f'anomaly_{method}'] = anomalies

# Analyze anomalies by procedure type
print(f"\n🔬 ANOMALY ANALYSIS BY PROCEDURE TYPE")
print("=" * 50)

anomaly_by_procedure = df_clean[df_clean['anomaly_consensus']].groupby('procedure_name').agg({
    'claim_id': 'count',
    'total_billed_amount': ['mean', 'max'],
    'insurance_paid_amount': 'mean'
}).round(2)

anomaly_by_procedure.columns = ['anomaly_count', 'avg_amount', 'max_amount', 'avg_insurance_paid']
anomaly_by_procedure = anomaly_by_procedure.sort_values('anomaly_count', ascending=False)
print(anomaly_by_procedure)

# Analyze anomalies by provider
print(f"\n👨‍⚕️ TOP PROVIDERS WITH ANOMALIES")
print("=" * 40)

provider_anomalies = df_clean[df_clean['anomaly_consensus']].groupby('provider_id').agg({
    'claim_id': 'count',
    'total_billed_amount': 'sum',
    'patient_id': 'nunique'
}).sort_values('claim_id', ascending=False).head(10)

provider_anomalies.columns = ['anomaly_claims', 'total_anomaly_amount', 'unique_patients']
print(provider_anomalies)

# Temporal analysis of anomalies
print(f"\n📅 TEMPORAL PATTERN OF ANOMALIES")
print("=" * 40)

temporal_anomalies = df_clean[df_clean['anomaly_consensus']].groupby('service_month').agg({
    'claim_id': 'count',
    'total_billed_amount': 'sum'
}).sort_values('service_month')

temporal_anomalies.columns = ['monthly_anomaly_count', 'monthly_anomaly_amount']
print(temporal_anomalies.tail(10))

print(f"\n✅ Statistical anomaly detection completed successfully!")

## 6. Machine Learning-Based Anomaly Detection 🤖

Applying advanced machine learning algorithms for multivariate anomaly detection using Isolation Forest, Local Outlier Factor, and One-Class SVM.

In [ ]:
def prepare_ml_features(df):
    """
    Prepare features for machine learning anomaly detection
    """
    print("🔧 PREPARING FEATURES FOR ML ANOMALY DETECTION")
    print("=" * 55)
    
    # Select numerical features for ML models
    numerical_features = [
        'patient_age', 'total_billed_amount', 'insurance_paid_amount',
        'patient_responsibility', 'length_of_stay', 'cost_per_day',
        'payment_rate', 'days_since_service', 'insurance_efficiency',
        'monthly_patient_total', 'provider_claim_count', 'provider_avg_billing'
    ]
    
    # Select categorical features to encode
    categorical_features = [
        'procedure_name', 'insurance_provider', 'department',
        'admission_type', 'patient_gender', 'age_group', 'cost_category'
    ]
    
    # Create feature matrix
    feature_df = df[numerical_features].copy()
    
    # One-hot encode categorical features (top categories only to avoid dimensionality explosion)
    for cat_col in categorical_features:
        # Get top 5 categories for each categorical feature
        top_categories = df[cat_col].value_counts().head(5).index
        for category in top_categories:
            feature_df[f'{cat_col}_{category}'] = (df[cat_col] == category).astype(int)
    
    # Handle any missing values
    feature_df = feature_df.fillna(feature_df.median())
    
    print(f"✅ Feature matrix prepared: {feature_df.shape}")
    print(f"Features included: {list(feature_df.columns)}")
    
    return feature_df

def apply_ml_anomaly_detection(features_df, contamination=0.05):
    """
    Apply multiple ML algorithms for anomaly detection
    """
    print(f"\n🤖 APPLYING ML ANOMALY DETECTION")
    print("=" * 40)
    print(f"Expected contamination rate: {contamination*100:.1f}%")
    
    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_df)
    
    ml_anomaly_results = {}
    
    # 1. Isolation Forest
    print(f"\n🌲 Isolation Forest:")
    iso_forest = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_estimators=100
    )
    iso_predictions = iso_forest.fit_predict(features_scaled)
    iso_anomalies = iso_predictions == -1
    ml_anomaly_results['isolation_forest'] = iso_anomalies
    
    print(f"   Anomalies detected: {iso_anomalies.sum():,} ({iso_anomalies.mean()*100:.2f}%)")
    print(f"   Anomaly scores range: {iso_forest.decision_function(features_scaled).min():.3f} to {iso_forest.decision_function(features_scaled).max():.3f}")
    
    # 2. Local Outlier Factor
    print(f"\n🎯 Local Outlier Factor:")
    lof = LocalOutlierFactor(
        contamination=contamination,
        n_neighbors=20
    )
    lof_predictions = lof.fit_predict(features_scaled)
    lof_anomalies = lof_predictions == -1
    ml_anomaly_results['local_outlier_factor'] = lof_anomalies
    
    print(f"   Anomalies detected: {lof_anomalies.sum():,} ({lof_anomalies.mean()*100:.2f}%)")
    print(f"   Outlier scores range: {lof.negative_outlier_factor_.min():.3f} to {lof.negative_outlier_factor_.max():.3f}")
    
    # 3. One-Class SVM
    print(f"\n⚡ One-Class SVM:")
    oc_svm = OneClassSVM(
        nu=contamination,
        kernel='rbf',
        gamma='scale'
    )
    svm_predictions = oc_svm.fit_predict(features_scaled)
    svm_anomalies = svm_predictions == -1
    ml_anomaly_results['one_class_svm'] = svm_anomalies
    
    print(f"   Anomalies detected: {svm_anomalies.sum():,} ({svm_anomalies.mean()*100:.2f}%)")
    
    # Ensemble approach - majority voting
    anomaly_counts = sum(ml_anomaly_results.values())
    ensemble_anomalies = anomaly_counts >= 2  # At least 2 models agree
    ml_anomaly_results['ml_ensemble'] = ensemble_anomalies
    
    print(f"\n🤝 ML Ensemble (≥2 models agree):")
    print(f"   Anomalies detected: {ensemble_anomalies.sum():,} ({ensemble_anomalies.mean()*100:.2f}%)")
    
    return ml_anomaly_results, scaler

# Prepare features and apply ML anomaly detection
features_df = prepare_ml_features(df_clean)
ml_anomalies, feature_scaler = apply_ml_anomaly_detection(features_df, contamination=0.05)

# Add ML anomaly flags to dataframe
for method, anomalies in ml_anomalies.items():
    df_clean[f'ml_anomaly_{method}'] = anomalies

# Compare ML methods with statistical methods
print(f"\n📊 COMPARISON: STATISTICAL vs ML METHODS")
print("=" * 50)

comparison_df = pd.DataFrame({
    'Method': ['Z-Score', 'IQR', 'Percentile', 'Modified Z-Score', 'Statistical Consensus',
               'Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'ML Ensemble'],
    'Anomalies_Detected': [
        df_clean['anomaly_z_score'].sum(),
        df_clean['anomaly_iqr'].sum(),
        df_clean['anomaly_percentile'].sum(),
        df_clean['anomaly_modified_z'].sum(),
        df_clean['anomaly_consensus'].sum(),
        df_clean['ml_anomaly_isolation_forest'].sum(),
        df_clean['ml_anomaly_local_outlier_factor'].sum(),
        df_clean['ml_anomaly_one_class_svm'].sum(),
        df_clean['ml_anomaly_ml_ensemble'].sum()
    ]
})
comparison_df['Percentage'] = (comparison_df['Anomalies_Detected'] / len(df_clean) * 100).round(2)
print(comparison_df)

# Analyze overlap between statistical and ML methods
print(f"\n🔄 METHOD OVERLAP ANALYSIS")
print("=" * 30)

stat_anomalies = df_clean['anomaly_consensus']
ml_ensemble_anomalies = df_clean['ml_anomaly_ml_ensemble']

overlap = (stat_anomalies & ml_ensemble_anomalies).sum()
only_statistical = (stat_anomalies & ~ml_ensemble_anomalies).sum()
only_ml = (~stat_anomalies & ml_ensemble_anomalies).sum()

print(f"Both methods agree: {overlap:,} ({overlap/len(df_clean)*100:.2f}%)")
print(f"Only statistical: {only_statistical:,} ({only_statistical/len(df_clean)*100:.2f}%)")
print(f"Only ML methods: {only_ml:,} ({only_ml/len(df_clean)*100:.2f}%)")

# Create final anomaly flag combining best of both approaches
df_clean['final_anomaly_flag'] = (
    df_clean['anomaly_consensus'] | df_clean['ml_anomaly_ml_ensemble']
)

final_anomaly_count = df_clean['final_anomaly_flag'].sum()
print(f"\nFinal anomaly detection: {final_anomaly_count:,} anomalies ({final_anomaly_count/len(df_clean)*100:.2f}%)")

print(f"\n✅ Machine learning anomaly detection completed successfully!")

## 7. Cost Pattern Analysis 💰

Deep dive analysis of cost patterns, trends, and relationships to identify billing insights and business intelligence.

In [ ]:
# Cost Analysis by Different Dimensions

print("💰 COMPREHENSIVE COST PATTERN ANALYSIS")
print("=" * 50)

# 1. Cost by Procedure Type
print("\n📋 COST ANALYSIS BY PROCEDURE TYPE")
print("=" * 40)

procedure_analysis = df_clean.groupby('procedure_name').agg({
    'total_billed_amount': ['count', 'mean', 'median', 'sum', 'std'],
    'insurance_paid_amount': 'mean',
    'patient_responsibility': 'mean',
    'length_of_stay': 'mean',
    'final_anomaly_flag': 'sum'
}).round(2)

procedure_analysis.columns = [
    'claim_count', 'avg_cost', 'median_cost', 'total_revenue', 'cost_std',
    'avg_insurance_paid', 'avg_patient_responsibility', 'avg_length_stay', 'anomaly_count'
]

procedure_analysis['anomaly_rate'] = (
    procedure_analysis['anomaly_count'] / procedure_analysis['claim_count'] * 100
).round(2)

procedure_analysis = procedure_analysis.sort_values('total_revenue', ascending=False)
print(procedure_analysis)

# 2. Cost by Insurance Provider
print(f"\n🏥 COST ANALYSIS BY INSURANCE PROVIDER")
print("=" * 45)

insurance_analysis = df_clean.groupby('insurance_provider').agg({
    'total_billed_amount': ['count', 'mean', 'sum'],
    'insurance_paid_amount': 'mean',
    'payment_rate': 'mean',
    'final_anomaly_flag': 'sum'
}).round(2)

insurance_analysis.columns = [
    'claim_count', 'avg_billing', 'total_billing', 'avg_payment', 'avg_payment_rate', 'anomaly_count'
]

insurance_analysis['anomaly_rate'] = (
    insurance_analysis['anomaly_count'] / insurance_analysis['claim_count'] * 100
).round(2)

insurance_analysis = insurance_analysis.sort_values('total_billing', ascending=False)
print(insurance_analysis)

# 3. Cost by Department
print(f"\n🏢 COST ANALYSIS BY DEPARTMENT")
print("=" * 35)

department_analysis = df_clean.groupby('department').agg({
    'total_billed_amount': ['count', 'mean', 'sum'],
    'provider_id': 'nunique',
    'patient_id': 'nunique',
    'final_anomaly_flag': 'sum'
}).round(2)

department_analysis.columns = [
    'claim_count', 'avg_billing', 'total_billing', 'unique_providers', 'unique_patients', 'anomaly_count'
]

department_analysis = department_analysis.sort_values('total_billing', ascending=False)
print(department_analysis)

# 4. Temporal Cost Analysis
print(f"\n📅 TEMPORAL COST TRENDS")
print("=" * 25)

# Monthly trends
monthly_trends = df_clean.groupby('service_month').agg({
    'total_billed_amount': ['count', 'sum', 'mean'],
    'final_anomaly_flag': 'sum'
}).round(2)

monthly_trends.columns = ['monthly_claims', 'monthly_revenue', 'avg_claim_amount', 'monthly_anomalies']
monthly_trends['anomaly_rate'] = (
    monthly_trends['monthly_anomalies'] / monthly_trends['monthly_claims'] * 100
).round(2)

print("Recent 6 months:")
print(monthly_trends.tail(6))

# 5. Provider Performance Analysis
print(f"\n👨‍⚕️ TOP PROVIDER PERFORMANCE ANALYSIS")
print("=" * 45)

# Top providers by revenue
top_providers = df_clean.groupby('provider_id').agg({
    'total_billed_amount': ['count', 'sum', 'mean'],
    'patient_id': 'nunique',
    'final_anomaly_flag': 'sum'
}).round(2)

top_providers.columns = ['total_claims', 'total_revenue', 'avg_claim_amount', 'unique_patients', 'anomaly_count']
top_providers['anomaly_rate'] = (
    top_providers['anomaly_count'] / top_providers['total_claims'] * 100
).round(2)
top_providers['revenue_per_patient'] = (
    top_providers['total_revenue'] / top_providers['unique_patients']
).round(2)

# Filter providers with significant volume (>50 claims)
significant_providers = top_providers[top_providers['total_claims'] >= 50]
print("Top 10 providers by revenue (minimum 50 claims):")
print(significant_providers.sort_values('total_revenue', ascending=False).head(10))

# 6. Patient Demographics and Cost Analysis
print(f"\n👥 DEMOGRAPHIC COST ANALYSIS")
print("=" * 35)

# Age group analysis
age_analysis = df_clean.groupby('age_group').agg({
    'total_billed_amount': ['count', 'mean', 'sum'],
    'insurance_paid_amount': 'mean',
    'final_anomaly_flag': 'sum'
}).round(2)

age_analysis.columns = ['claim_count', 'avg_cost', 'total_cost', 'avg_insurance_paid', 'anomaly_count']
age_analysis['anomaly_rate'] = (age_analysis['anomaly_count'] / age_analysis['claim_count'] * 100).round(2)

print("Cost analysis by age group:")
print(age_analysis)

# Gender analysis
gender_analysis = df_clean.groupby('patient_gender').agg({
    'total_billed_amount': ['count', 'mean', 'sum'],
    'final_anomaly_flag': 'sum'
}).round(2)

gender_analysis.columns = ['claim_count', 'avg_cost', 'total_cost', 'anomaly_count']
gender_analysis['anomaly_rate'] = (gender_analysis['anomaly_count'] / gender_analysis['claim_count'] * 100).round(2)

print(f"\nCost analysis by gender:")
print(gender_analysis)

# 7. High-Value Claims Analysis
print(f"\n💎 HIGH-VALUE CLAIMS ANALYSIS")
print("=" * 35)

# Define high-value threshold (top 5%)
high_value_threshold = df_clean['total_billed_amount'].quantile(0.95)
high_value_claims = df_clean[df_clean['total_billed_amount'] >= high_value_threshold]

print(f"High-value threshold: ${high_value_threshold:,.2f}")
print(f"High-value claims: {len(high_value_claims):,} ({len(high_value_claims)/len(df_clean)*100:.2f}%)")
print(f"Revenue from high-value claims: ${high_value_claims['total_billed_amount'].sum():,.2f}")
print(f"Percentage of total revenue: {high_value_claims['total_billed_amount'].sum()/df_clean['total_billed_amount'].sum()*100:.1f}%")

high_value_summary = high_value_claims.groupby(['procedure_name', 'department']).agg({
    'claim_id': 'count',
    'total_billed_amount': ['mean', 'max'],
    'final_anomaly_flag': 'sum'
}).round(2)

print(f"\nHigh-value claims by procedure and department:")
print(high_value_summary.head(10))

print(f"\n✅ Cost pattern analysis completed successfully!")

## 8. Visualization of Anomalies and Insights 📊

Creating comprehensive visualizations to display anomalies, cost patterns, and business insights for stakeholder communication.

In [ ]:
# Set up plotting parameters
plt.style.use('default')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

print("📊 CREATING COMPREHENSIVE VISUALIZATIONS")
print("=" * 50)

# 1. Anomaly Detection Overview
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Anomaly distribution by method
anomaly_methods = ['Z-Score', 'IQR', 'Percentile', 'Statistical Consensus', 
                   'Isolation Forest', 'LOF', 'One-Class SVM', 'ML Ensemble', 'Final Combined']
anomaly_counts = [
    df_clean['anomaly_z_score'].sum(),
    df_clean['anomaly_iqr'].sum(), 
    df_clean['anomaly_percentile'].sum(),
    df_clean['anomaly_consensus'].sum(),
    df_clean['ml_anomaly_isolation_forest'].sum(),
    df_clean['ml_anomaly_local_outlier_factor'].sum(),
    df_clean['ml_anomaly_one_class_svm'].sum(),
    df_clean['ml_anomaly_ml_ensemble'].sum(),
    df_clean['final_anomaly_flag'].sum()
]

axes[0,0].bar(range(len(anomaly_methods)), anomaly_counts, color=colors)
axes[0,0].set_xticks(range(len(anomaly_methods)))
axes[0,0].set_xticklabels(anomaly_methods, rotation=45, ha='right')
axes[0,0].set_title('Anomalies Detected by Different Methods', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Number of Anomalies')

# Cost distribution with anomalies highlighted
normal_costs = df_clean[~df_clean['final_anomaly_flag']]['total_billed_amount']
anomaly_costs = df_clean[df_clean['final_anomaly_flag']]['total_billed_amount']

axes[0,1].hist(normal_costs, bins=50, alpha=0.7, label='Normal Claims', color='lightblue', density=True)
axes[0,1].hist(anomaly_costs, bins=50, alpha=0.7, label='Anomalous Claims', color='red', density=True)
axes[0,1].set_xlabel('Total Billed Amount ($)')
axes[0,1].set_ylabel('Density')
axes[0,1].set_title('Cost Distribution: Normal vs Anomalous Claims', fontsize=14, fontweight='bold')
axes[0,1].legend()
axes[0,1].set_xlim(0, 20000)  # Focus on main distribution

# Anomalies by procedure type
procedure_anomalies = df_clean[df_clean['final_anomaly_flag']].groupby('procedure_name').size().sort_values(ascending=True)
axes[1,0].barh(range(len(procedure_anomalies)), procedure_anomalies.values, color='coral')
axes[1,0].set_yticks(range(len(procedure_anomalies)))
axes[1,0].set_yticklabels(procedure_anomalies.index)
axes[1,0].set_title('Anomalies by Procedure Type', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Number of Anomalies')

# Temporal anomaly pattern
monthly_anomalies = df_clean[df_clean['final_anomaly_flag']].groupby('service_month').size()
axes[1,1].plot(range(len(monthly_anomalies)), monthly_anomalies.values, marker='o', linewidth=2, markersize=6, color='darkred')
axes[1,1].set_xticks(range(0, len(monthly_anomalies), 3))
axes[1,1].set_xticklabels([monthly_anomalies.index[i] for i in range(0, len(monthly_anomalies), 3)], rotation=45)
axes[1,1].set_title('Monthly Anomaly Trends', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Number of Anomalies')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 2. Cost Pattern Analysis Visualizations
fig, axes = plt.subplots(2, 2, figsize=(20, 16))

# Average cost by procedure
procedure_costs = df_clean.groupby('procedure_name')['total_billed_amount'].mean().sort_values(ascending=True)
axes[0,0].barh(range(len(procedure_costs)), procedure_costs.values, color='skyblue')
axes[0,0].set_yticks(range(len(procedure_costs)))
axes[0,0].set_yticklabels(procedure_costs.index)
axes[0,0].set_title('Average Cost by Procedure Type', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Average Cost ($)')

# Insurance payment efficiency
insurance_efficiency = df_clean.groupby('insurance_provider')['payment_rate'].mean().sort_values(ascending=False)
axes[0,1].bar(range(len(insurance_efficiency)), insurance_efficiency.values, color='lightgreen')
axes[0,1].set_xticks(range(len(insurance_efficiency)))
axes[0,1].set_xticklabels(insurance_efficiency.index, rotation=45, ha='right')
axes[0,1].set_title('Insurance Payment Efficiency', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Average Payment Rate')
axes[0,1].set_ylim(0, 1)

# Cost by age group
age_costs = df_clean.groupby('age_group')['total_billed_amount'].mean()
age_order = ['Pediatric (0-17)', 'Young Adult (18-34)', 'Middle Age (35-49)', 'Older Adult (50-64)', 'Senior (65+)']
age_costs_ordered = [age_costs[age] for age in age_order if age in age_costs.index]
axes[1,0].bar(range(len(age_costs_ordered)), age_costs_ordered, color='orange')
axes[1,0].set_xticks(range(len(age_costs_ordered)))
axes[1,0].set_xticklabels([age for age in age_order if age in age_costs.index], rotation=45, ha='right')
axes[1,0].set_title('Average Cost by Age Group', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Average Cost ($)')

# Monthly revenue trends
monthly_revenue = df_clean.groupby('service_month')['total_billed_amount'].sum()
axes[1,1].plot(range(len(monthly_revenue)), monthly_revenue.values, marker='s', linewidth=3, markersize=8, color='purple')
axes[1,1].set_xticks(range(0, len(monthly_revenue), 3))
axes[1,1].set_xticklabels([monthly_revenue.index[i] for i in range(0, len(monthly_revenue), 3)], rotation=45)
axes[1,1].set_title('Monthly Revenue Trends', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Total Revenue ($)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3. Interactive Plotly Visualizations
print("\n🎨 Creating Interactive Plotly Visualizations...")

# Interactive scatter plot: Cost vs Age with anomalies highlighted
fig_scatter = px.scatter(
    df_clean.sample(5000),  # Sample for performance
    x='patient_age',
    y='total_billed_amount',
    color='final_anomaly_flag',
    hover_data=['procedure_name', 'insurance_provider', 'department'],
    title='Healthcare Costs by Patient Age (Anomalies Highlighted)',
    labels={'patient_age': 'Patient Age', 'total_billed_amount': 'Total Billed Amount ($)'}
)
fig_scatter.update_layout(height=600)
fig_scatter.show()

# Box plot of costs by procedure type
fig_box = px.box(
    df_clean,
    x='procedure_name',
    y='total_billed_amount',
    title='Cost Distribution by Procedure Type',
    labels={'procedure_name': 'Procedure Type', 'total_billed_amount': 'Total Billed Amount ($)'}
)
fig_box.update_xaxes(tickangle=45)
fig_box.update_layout(height=600)
fig_box.show()

# Heatmap of anomaly rates by procedure and department
anomaly_heatmap = df_clean.groupby(['procedure_name', 'department']).agg({
    'final_anomaly_flag': 'mean',
    'claim_id': 'count'
}).reset_index()
anomaly_heatmap = anomaly_heatmap[anomaly_heatmap['claim_id'] >= 10]  # Filter for significance

heatmap_pivot = anomaly_heatmap.pivot(index='procedure_name', columns='department', values='final_anomaly_flag')

fig_heatmap = px.imshow(
    heatmap_pivot,
    title='Anomaly Rate Heatmap: Procedure Type vs Department',
    labels={'color': 'Anomaly Rate'},
    aspect='auto'
)
fig_heatmap.update_layout(height=600)
fig_heatmap.show()

# 4. Provider Performance Analysis Visualization
print("\n👨‍⚕️ Provider Performance Visualizations...")

# Top providers by anomaly rate (minimum 30 claims)
provider_performance = df_clean.groupby('provider_id').agg({
    'total_billed_amount': ['count', 'sum', 'mean'],
    'final_anomaly_flag': 'sum'
}).round(2)

provider_performance.columns = ['claim_count', 'total_revenue', 'avg_claim_amount', 'anomaly_count']
provider_performance['anomaly_rate'] = (
    provider_performance['anomaly_count'] / provider_performance['claim_count']
).round(3)

# Filter for providers with sufficient volume
qualified_providers = provider_performance[provider_performance['claim_count'] >= 30]
high_anomaly_providers = qualified_providers.nlargest(15, 'anomaly_rate')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# High anomaly rate providers
ax1.barh(range(len(high_anomaly_providers)), high_anomaly_providers['anomaly_rate'], color='red', alpha=0.7)
ax1.set_yticks(range(len(high_anomaly_providers)))
ax1.set_yticklabels(high_anomaly_providers.index)
ax1.set_title('Providers with Highest Anomaly Rates (≥30 claims)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Anomaly Rate')

# Provider revenue vs anomaly rate scatter
ax2.scatter(qualified_providers['total_revenue'], qualified_providers['anomaly_rate'], 
           alpha=0.6, s=qualified_providers['claim_count']*2, color='blue')
ax2.set_xlabel('Total Revenue ($)')
ax2.set_ylabel('Anomaly Rate')
ax2.set_title('Provider Revenue vs Anomaly Rate\n(Bubble size = Claim Count)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ All visualizations created successfully!")
print("📊 Interactive plots displayed above provide detailed insights for stakeholder analysis.")

## 9. Export Results for Power BI Integration 📤

Preparing and exporting cleaned data, anomaly flags, and summary statistics in formats optimized for Power BI dashboard creation.

In [ ]:
# Prepare data exports for Power BI Dashboard
import os

# Create exports directory if it doesn't exist
export_dir = '../data/powerbi_exports'
os.makedirs(export_dir, exist_ok=True)

print("📤 PREPARING DATA EXPORTS FOR POWER BI")
print("=" * 50)

# 1. Main Dataset with Anomaly Flags
print("1️⃣ Exporting main dataset with anomaly flags...")

# Select key columns for Power BI
powerbi_columns = [
    'claim_id', 'patient_id', 'patient_age', 'patient_gender', 'age_group',
    'service_date', 'service_month', 'service_quarter', 'service_year', 'day_of_week',
    'procedure_name', 'procedure_code', 'procedure_complexity', 'primary_diagnosis',
    'department', 'provider_id', 'insurance_provider', 'admission_type', 'facility_type',
    'total_billed_amount', 'insurance_paid_amount', 'patient_responsibility',
    'payment_rate', 'insurance_efficiency', 'length_of_stay', 'cost_per_day',
    'cost_category', 'claim_status', 'final_anomaly_flag', 'is_potential_duplicate',
    'monthly_patient_total', 'provider_avg_billing', 'provider_claim_count'
]

powerbi_main = df_clean[powerbi_columns].copy()

# Add readable anomaly status
powerbi_main['anomaly_status'] = powerbi_main['final_anomaly_flag'].map({True: 'Anomalous', False: 'Normal'})

# Export main dataset
powerbi_main.to_csv(f'{export_dir}/healthcare_billing_main.csv', index=False)
print(f"   ✅ Exported: healthcare_billing_main.csv ({len(powerbi_main):,} records)")

# 2. Anomaly Summary Table
print("\n2️⃣ Creating anomaly summary table...")

anomaly_summary = pd.DataFrame({
    'Detection_Method': [
        'Z-Score', 'IQR', 'Percentile', 'Modified Z-Score', 'Statistical Consensus',
        'Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'ML Ensemble', 'Final Combined'
    ],
    'Anomalies_Detected': [
        df_clean['anomaly_z_score'].sum(),
        df_clean['anomaly_iqr'].sum(),
        df_clean['anomaly_percentile'].sum(),
        df_clean['anomaly_modified_z'].sum(),
        df_clean['anomaly_consensus'].sum(),
        df_clean['ml_anomaly_isolation_forest'].sum(),
        df_clean['ml_anomaly_local_outlier_factor'].sum(),
        df_clean['ml_anomaly_one_class_svm'].sum(),
        df_clean['ml_anomaly_ml_ensemble'].sum(),
        df_clean['final_anomaly_flag'].sum()
    ]
})
anomaly_summary['Percentage'] = (anomaly_summary['Anomalies_Detected'] / len(df_clean) * 100).round(2)
anomaly_summary['Method_Type'] = ['Statistical'] * 5 + ['Machine Learning'] * 4 + ['Combined']

anomaly_summary.to_csv(f'{export_dir}/anomaly_detection_summary.csv', index=False)
print(f"   ✅ Exported: anomaly_detection_summary.csv")

# 3. Procedure Analysis Summary
print("\n3️⃣ Creating procedure analysis summary...")

procedure_summary = df_clean.groupby(['procedure_name', 'procedure_complexity']).agg({
    'claim_id': 'count',
    'total_billed_amount': ['mean', 'median', 'sum', 'std'],
    'insurance_paid_amount': 'mean',
    'length_of_stay': 'mean',
    'final_anomaly_flag': 'sum',
    'patient_id': 'nunique'
}).round(2)

procedure_summary.columns = [
    'total_claims', 'avg_cost', 'median_cost', 'total_revenue', 'cost_std',
    'avg_insurance_paid', 'avg_length_stay', 'anomaly_count', 'unique_patients'
]

procedure_summary['anomaly_rate'] = (
    procedure_summary['anomaly_count'] / procedure_summary['total_claims'] * 100
).round(2)

procedure_summary = procedure_summary.reset_index()
procedure_summary.to_csv(f'{export_dir}/procedure_analysis_summary.csv', index=False)
print(f"   ✅ Exported: procedure_analysis_summary.csv")

# 4. Provider Performance Summary
print("\n4️⃣ Creating provider performance summary...")

provider_summary = df_clean.groupby('provider_id').agg({
    'claim_id': 'count',
    'total_billed_amount': ['sum', 'mean'],
    'patient_id': 'nunique',
    'final_anomaly_flag': 'sum',
    'department': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
}).round(2)

provider_summary.columns = [
    'total_claims', 'total_revenue', 'avg_claim_amount', 'unique_patients', 
    'anomaly_count', 'primary_department'
]

provider_summary['anomaly_rate'] = (
    provider_summary['anomaly_count'] / provider_summary['total_claims'] * 100
).round(2)

provider_summary['revenue_per_patient'] = (
    provider_summary['total_revenue'] / provider_summary['unique_patients']
).round(2)

provider_summary = provider_summary.reset_index()

# Add performance categories
def categorize_performance(row):
    if row['anomaly_rate'] > 10:
        return 'High Risk'
    elif row['anomaly_rate'] > 5:
        return 'Medium Risk'
    else:
        return 'Low Risk'

provider_summary['risk_category'] = provider_summary.apply(categorize_performance, axis=1)

provider_summary.to_csv(f'{export_dir}/provider_performance_summary.csv', index=False)
print(f"   ✅ Exported: provider_performance_summary.csv")

# 5. Monthly Trends Summary
print("\n5️⃣ Creating monthly trends summary...")

monthly_summary = df_clean.groupby(['service_month', 'service_year', 'service_quarter']).agg({
    'claim_id': 'count',
    'total_billed_amount': ['sum', 'mean'],
    'insurance_paid_amount': 'sum',
    'patient_responsibility': 'sum',
    'final_anomaly_flag': 'sum',
    'patient_id': 'nunique',
    'provider_id': 'nunique'
}).round(2)

monthly_summary.columns = [
    'monthly_claims', 'monthly_revenue', 'avg_claim_amount', 'monthly_insurance_paid',
    'monthly_patient_responsibility', 'monthly_anomalies', 'unique_patients', 'unique_providers'
]

monthly_summary['anomaly_rate'] = (
    monthly_summary['monthly_anomalies'] / monthly_summary['monthly_claims'] * 100
).round(2)

monthly_summary = monthly_summary.reset_index()
monthly_summary.to_csv(f'{export_dir}/monthly_trends_summary.csv', index=False)
print(f"   ✅ Exported: monthly_trends_summary.csv")

# 6. Insurance Analysis Summary
print("\n6️⃣ Creating insurance analysis summary...")

insurance_summary = df_clean.groupby('insurance_provider').agg({
    'claim_id': 'count',
    'total_billed_amount': ['sum', 'mean'],
    'insurance_paid_amount': ['sum', 'mean'],
    'payment_rate': 'mean',
    'final_anomaly_flag': 'sum',
    'patient_id': 'nunique'
}).round(2)

insurance_summary.columns = [
    'total_claims', 'total_billed', 'avg_billed_per_claim', 'total_paid', 
    'avg_paid_per_claim', 'avg_payment_rate', 'anomaly_count', 'unique_patients'
]

insurance_summary['anomaly_rate'] = (
    insurance_summary['anomaly_count'] / insurance_summary['total_claims'] * 100
).round(2)

insurance_summary = insurance_summary.reset_index()
insurance_summary.to_csv(f'{export_dir}/insurance_analysis_summary.csv', index=False)
print(f"   ✅ Exported: insurance_analysis_summary.csv")

# 7. Key Performance Indicators (KPIs)
print("\n7️⃣ Creating KPI summary...")

kpi_summary = pd.DataFrame({
    'KPI_Name': [
        'Total Claims',
        'Total Revenue',
        'Average Claim Amount',
        'Total Anomalies',
        'Anomaly Rate (%)',
        'Unique Patients',
        'Unique Providers',
        'Average Length of Stay',
        'Average Payment Rate',
        'High Value Claims (>95th percentile)',
        'Potential Duplicates'
    ],
    'Value': [
        len(df_clean),
        df_clean['total_billed_amount'].sum(),
        df_clean['total_billed_amount'].mean(),
        df_clean['final_anomaly_flag'].sum(),
        df_clean['final_anomaly_flag'].mean() * 100,
        df_clean['patient_id'].nunique(),
        df_clean['provider_id'].nunique(),
        df_clean['length_of_stay'].mean(),
        df_clean['payment_rate'].mean(),
        len(df_clean[df_clean['total_billed_amount'] > df_clean['total_billed_amount'].quantile(0.95)]),
        df_clean['is_potential_duplicate'].sum()
    ]
})

kpi_summary['Formatted_Value'] = kpi_summary.apply(lambda row: 
    f"${row['Value']:,.2f}" if 'Revenue' in row['KPI_Name'] or 'Amount' in row['KPI_Name'] 
    else f"{row['Value']:,.0f}" if row['Value'] > 1 
    else f"{row['Value']:.2f}", axis=1)

kpi_summary.to_csv(f'{export_dir}/kpi_summary.csv', index=False)
print(f"   ✅ Exported: kpi_summary.csv")

# 8. Create Data Dictionary
print("\n8️⃣ Creating data dictionary...")

data_dictionary = pd.DataFrame({
    'Column_Name': powerbi_main.columns,
    'Data_Type': [str(dtype) for dtype in powerbi_main.dtypes],
    'Description': [
        'Unique claim identifier',
        'Unique patient identifier', 
        'Patient age in years',
        'Patient gender',
        'Age group category',
        'Date of service',
        'Service month (YYYY-MM)',
        'Service quarter (1-4)',
        'Service year',
        'Day of week for service',
        'Medical procedure name',
        'Medical procedure code',
        'Procedure complexity level',
        'Primary diagnosis code',
        'Hospital department',
        'Healthcare provider ID',
        'Insurance company name',
        'Type of admission',
        'Healthcare facility type',
        'Total amount billed',
        'Amount paid by insurance',
        'Amount owed by patient',
        'Insurance payment rate (0-1)',
        'Insurance coverage efficiency',
        'Length of hospital stay in days',
        'Cost per day of stay',
        'Cost category classification',
        'Claim processing status',
        'Final anomaly detection flag',
        'Potential duplicate claim flag',
        'Monthly billing total per patient',
        'Provider average billing amount',
        'Provider total claim count',
        'Anomaly status (Anomalous/Normal)'
    ]
})

data_dictionary.to_csv(f'{export_dir}/data_dictionary.csv', index=False)
print(f"   ✅ Exported: data_dictionary.csv")

# Summary of exports
print(f"\n📋 EXPORT SUMMARY")
print("=" * 25)
print(f"Export directory: {export_dir}")
print(f"Files created: 8")
print(f"Main dataset records: {len(powerbi_main):,}")
print(f"Date range: {df_clean['service_date'].min().date()} to {df_clean['service_date'].max().date()}")
print(f"Total anomalies flagged: {df_clean['final_anomaly_flag'].sum():,}")

print(f"\n✅ All exports completed successfully!")
print(f"🎯 Data is now ready for Power BI dashboard creation!")
print(f"💡 Use the KPI summary and data dictionary to build meaningful visualizations.")